In [1]:
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
import matplotlib.pyplot as plt

In [2]:
DIR = os.path.expanduser("~") + "/Dropbox/lml/genome_measurements/"

In [3]:
allgloss = dirwalk(DIR,require='los',omit=['legacy','flag','Test'])

In [4]:
df = pd.DataFrame()
for f in allgloss:
    tmp = fproc(f,dimension='gloss',threshold=0.5)
    if tmp is not None:
        df = pd.concat([df,tmp])

In [5]:
df = df.drop_duplicates(subset=df.columns.difference(['gloss','fname']))

In [6]:
df = df.reset_index(drop=True)

In [7]:
df['mera'] = 'mod'

In [8]:
df['g60'] = df.gloss.map(lambda x: x[1])

# Legacy

In [9]:
legacygloss = dirwalk(DIR,require=['los','egac'])
legacygloss[0]

'/Users/damoncrockett/Dropbox/lml/genome_measurements/genome_legacy/gloss.xlsx'

We will use only this file, with measurements from 2015; Genevieve did a bunch of tests on binder papers in 2018, but none are at the 60 degree angle.

In [10]:
paul = fproc(legacygloss[0],dimension='gloss')

In [11]:
paul = paul[paul['gloss'].apply(lambda x: len(x)) != 11] # we don't have a 60 degree mean for this item

In [12]:
# If there are 12 measurements, we want the 5th; if there are 3, we want the 2nd

paul['g60'] = paul.gloss.map(lambda x: x[4] if len(x)==12 else x[2])

In [13]:
paul = paul[['catalog','fname','g60']]

In [14]:
#paul.catalog.value_counts().loc[lambda x: x > 1]

# STD

In [15]:
allvals = list(df.g60) + list(paul.g60)

In [16]:
g60std = np.std(allvals)
g60std

30.15660308539135

# Enumeration of Cases

This will be different from the thickness case, because here, our trials are spread across the rows, not the columns. That actually makes things more difficult, especially since the sample IDs are not 100% consistent.

#### Catalog extract

In [17]:
df['cat'] = df.catalog.apply(lambda x: parse_sample_id(x)['catalog'])
df['manbran'] = df.catalog.apply(lambda x: parse_sample_id(x)['manbran'])
df['mtrial'] = df.catalog.apply(lambda x: parse_sample_id(x)['mtrial'])

# Mtrial corrections 

I had meant to do this later in the process, but these corrections are very few and easy to make and will make it easier to assess duplicate cases.

In [18]:
df.mtrial[df.catalog=='2484_kb_'] = '0'
df.mtrial[df.catalog=='2484_kb_001'] = '1'
df.mtrial[df.catalog=='2484_kb_002'] = '2'

In [19]:
df.mtrial[df.catalog=='5515y_al_30'] = '3'

In [20]:
df = df[df.catalog!='Average']

In [21]:
df = df.reset_index(drop=True)

In [22]:
pkl(df,'raw.pkl')
pkl(paul,'lraw.pkl')